In [6]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import sys
import numpy as np
import pandas as pd
import regex as re
import requests
import lxml
from lxml.html.soupparser import fromstring
import prettify
import numbers
import htmltext

In [7]:
#add headers in case you use chromedriver (captchas are no fun); namely used for chromedriver
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

In [67]:
def zillow_web(city,pages):
    
    # Adding headers for web-scrapping 
    req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
    }
    
    #creating empty List for url
    url_list=[]
    
    for x in range(pages):
        url_list.append(f'https://www.zillow.com/homes/for_sale/{city}/{x+1}_p/')
    
    #creating list of objects
    obj=[]
    
    for x in range(pages):
        hg= requests.get(url_list[x], headers=req_headers)
        obj.append(BeautifulSoup(hg.content, 'html.parser'))
        
    #create the first two dataframes
    df = pd.DataFrame()
    df1 = pd.DataFrame()

    #for loops is pulling the specified variable using beautiful soup and inserting into said variable
    
    for x in obj:
        for i in x:
            address = x.find_all (class_= 'list-card-addr')
            price = list(x.find_all (class_='list-card-price'))
            beds = list(x.find_all("ul", class_="list-card-details"))
            
        #create dataframe columns out of variables
        df1['prices'] = price
        df1['address'] = address
        df1['beds'] = beds  

        df = df.append(df1, ignore_index = True) 

        
    # segregating Beds column into beds, bath and squarefeet
    zillow_beds = df['beds']
  
    import re
    beds=[]
    baths=[]
    sqft=[]
    reg2 = "[^.,0-9]"
    reg1 = "\s+"
    
   
    for x in zillow_beds:
        string_ = re.sub(reg2," ",str(x))
        string_.lstrip()
        string_.rstrip()
        str1 = re.sub(reg1," ",string_)
        
        beds.append(str1.split(' ')[1])
        baths.append(str1.split(' ')[2])
        try:
            sqft.append(str1.split(' ')[3])
        except Exception as e:
            sqft.append("NA") 
        
    # Formatting Adddress column - 
    zillow_add = df['address']
    
    add=[]
    for x in zillow_add:
        a = str(x).split('<')[1]
        b = a.split('>')[1]
        add.append(b)
        
    # Formatting Price column - 
    zillow_price = df['prices']
    
    price=[]
    for x in zillow_price:
        a = str(x).split('<')[1]
        b = a.split('>')[1]
        price.append(b)
    
    # Creating the output dataset using all the modified columns - 
    
    output=pd.DataFrame()
    output["Price"]= price
    output["Address"]= add
    output["Beds"]=beds
    output["baths"]=baths
    output["sqft"]=sqft
    
    #Exporting the created dataset to csv
    output.to_csv('zillow_output.csv')
    #Exporting the raw dataset without modification to csv
    df.to_csv('zillow_raw.csv')
    
    return output


In [69]:
#Calling Function
zillow_web("18505",10)

,Price,Address,Beds,baths,sqft
0,"$155,000","904 Gibbons St, Scranton, PA 18505",3,2,"2,592"
1,"$399,900","1 Scranton Pocono Hwy, Scranton, PA 18505",5,5,"3,279"
2,"$115,000","417 Pittston Ave, Scranton, PA 18505",3,3,"1,900"
3,"$158,900","611 E Locust St, Scranton, PA 18505",3,2,"1,344"
4,"$119,000","424 S Webster Ave, Scranton, PA 18505",6,17,"2,000"
...,...,...,...,...,...
85,"$119,000","424 S Webster Ave, Scranton, PA 18505",6,17,"2,000"
86,"$74,900","1328 E Mountain Rd, Scranton, PA 18505",2,1,980
87,"$399,900","2530 Birney Ave, Scranton, PA 18505",3,3,"3,000"
88,"$175,000","1412 E Elm St, Scranton, PA 18505",4,2,"2,820"
